In [ ]:
pip install --upgrade pip

In [ ]:
# Install requirements
!pip install -r Requirements.txt

In [ ]:
# View AWS Configuration
!aws configure list

----Code Start----

In [ ]:
# Imports 
from ray.util import inspect_serializability
import ray
import pyarrow.fs as pq
import pandas as pd

In [ ]:
# Reusable definitions here
bucketName = 'DH-SECURE-THANOS-RAY-USE'
endpoint = 'https://s3.upshift.redhat.com'

prefixPathRead = 'raydev'
prefixPathWrite = 'raydev-write-demo'
metricName = 'cluster_version'

year = '2021'
month = '01'
day = '01'

Following cells reads one days worth of parquet files and writes back a single parquet file
Month 1, Day 1.
1. Fails currently because worker nodes keep dying.
2. Success! Successful compaction for a days worth of parquets. Next is a month.

In [ ]:
# Read path
read_path = f'{prefixPathRead}/metric={metricName}'
read_bucket_uri = f's3://{bucketName}/{read_path}/year={year}/month={month}/day={day}'
print(read_bucket_uri)

# Write path
write_path = f'{prefixPathWrite}/metric={metricName}'
write_bucket_uri = f's3://{bucketName}/{write_path}/year={year}/month={month}/day={day}'

In [ ]:
# S3FileSystem in PyArrow
fs_pyarrow = pq.S3FileSystem(endpoint_override=endpoint)

# Reading parquet using Ray through filesystem
df = ray.data.read_parquet(paths=read_bucket_uri, filesystem=fs_pyarrow)

# Writing back a single parquet file
df.repartition(1).write_parquet(path=write_bucket_uri, filesystem=fs_pyarrow)

Following cells try compaction for a whole month:
1. Reading from multiple directories seems to be not supported
2. Union of dataframes is supported, can be done before the compaction step. https://github.com/ray-project/ray/issues/24598
Status: Fails due to memory limits, cannot load a dataframe for more than several days worth of parquets

In [ ]:
# Read path
read_path = f'{prefixPathRead}/metric={metricName}'
read_bucket_uri = f's3://{bucketName}/{read_path}/year={year}/month={month}/day={day}'
# Holds paths 
read_bucket_multiple = []

# Creating paths for every day in the month.
for i in range(2,32):
    read_bucket_multiple.append(f"s3://{bucketName}/{read_path}/year={year}/month={month}/day="f"{i:02d}")
                           
#print(read_bucket_multiple)


# Write path
write_path = f"{prefixPath}/metric={metricName}"
write_bucket_uri = f's3://{bucketName}/{write_path}/year={year}/month={month}'


In [ ]:
# S3FileSystem in PyArrow
fs_pyarrow = pq.S3FileSystem(endpoint_override=endpoint)


# Initial read - First day to start a dataframe
master_df = ray.data.read_parquet(paths=f's3://{bucketName}/{prefixPathRead}/metric={metricName}/year={year}/month={month}/day=01', filesystem=fs_pyarrow)

# Reading remaining days
for x in read_bucket_multiple:
    df = ray.data.read_parquet(paths=x, filesystem=fs_pyarrow) # Reading parquets for each day using Ray through filesystem
    master_df = master_df.union(df) # Followed by unioning the dataframe each time
    


2022-06-14 14:17:12,492	WARNING worker.py:1382 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: 183fd5ac5279cea1f9ecb32047c9021ec68b455701000000 Worker ID: b80b93f419a8e18eb1ea4296629279bd8cfd387ab6a633649b309e70 Node ID: fedc7d52d35d622bb8e7ca123054f7ece21cf6306322dc515efc0848 Worker IP address: 172.44.16.44 Worker port: 46225 Worker PID: 41860
2022-06-14 14:17:17,390	WARNING worker.py:1382 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: 68c163d95417a4d8e0a6211cfdaa2f0ca48d163201000000 Worker ID: b2fe3d827c76c63b546683108646f53fb1463763ef4afbdfcd9dd668 Node ID: fedc7d52d35d622bb8e7ca123054f7ece21cf6306322dc515efc0848 Worker IP address: 172.44.16.44 Worker port: 40009 Worker PID: 41969
2022-06-14 14:17:22,559	WARNING worker.py:1382 -- A worker died or was killed 

In [ ]:
# Writing back a single parquet file
df.repartition(1).write_parquet(path=write_bucket_uri, filesystem=fs_pyarrow)